# Simplex Method
This is a project to showcase numerical methods and implementation of nontrivial algorithms. 

Methods like this are utilized as solutions to resource allocation and operation research problems.

*Note that no AI coding agents were used to write any code, however AI agents were queried for syntax questions.*

In [1]:
import numpy as np

In [3]:

class SimplexSolver(object):
    """Class for solving the standard linear optimization problem

                        minimize        c^Tx
                        subject to      Ax <= b
                                         x >= 0
    via the Simplex algorithm.
    """

    def __init__(self, c, A, b):
        """Check for feasibility and initialize the dictionary.

        Parameters:
            c ((n,) ndarray): The coefficients of the objective function.
            A ((m,n) ndarray): The constraint coefficients matrix.
            b ((m,) ndarray): The constraint vector.

        Raises:
            ValueError: if the given system is infeasible at the origin.
        """
        # Initialize Dimmensions
        dimensions = A.shape
        self.m = dimensions[0]
        self.n = dimensions[1]

        # Check feasability
        x_star = np.zeros(self.n)
        if (A@x_star > b).any():
            raise ValueError("The origin is not feasible.")
        
        # Initialize
        self.c = c
        self.A = A
        self.b = b

        # Initialize the dictionary
        self.dictionary = self._generatedictionary(self.c, self.A, self.b)
        


    def _generatedictionary(self, c, A, b):
        """Generate the initial dictionary.

        Parameters:
            c ((n,) ndarray): The coefficients of the objective function.
            A ((m,n) ndarray): The constraint coefficients matrix.
            b ((m,) ndarray): The constraint vector.
        """
        # Stacks a zero on top of the vertical of b
        first_col = np.vstack(([0], b.reshape(-1, 1)))

        # Horizontal Stacks A with the identity matrix
        A_bar = np.hstack((A, np.eye(self.m)))
        
        # Stacks c with m zeros vertically
        c_bar = np.hstack((c, np.zeros(self.m)))

        # Vertically stacks c_bar with -A_bar
        second_col = np.vstack((c_bar.T, -A_bar))

        # Return all stacked together
        return np.hstack((first_col, second_col))




    def _pivot_col(self):
        """Return the column index of the next pivot column.
        """
        # For each column find the first negative value
        for j, value in enumerate(self.dictionary[0, 1:]):
            if value < 0:
                return j + 1

        # If already solved on first go around (should never trigger because of while loop)
        else: 
            raise ValueError('No coefficent is negative')


    def _pivot_row(self, index):
        """Determine the row index of the next pivot row using the ratio test
        (Bland's Rule).
        """
        vals = []

        # Iterate through pivot column values to find the most negative one
        for j, value in enumerate(self.dictionary[1:, index]):
            if value < 0:
                vals.append(((-self.dictionary[j + 1, 0] / value), j))
        
        # Checks for all positive values
        if not vals:
            raise ValueError('Our problem is unbounded')
        
        # Returns the index of the most minimum value
        # Tie breakers are handled with the lambda function selecting the younger of the two
        return min(vals, key=lambda x: (x[0], x[1]))[1] + 1


    def pivot(self):
        """Select the column and row to pivot on. Reduce the column to a
        negative elementary vector.
        """
        # Get the pivot indexes
        pivot_col_index = self._pivot_col()
        pivot_row_index = self._pivot_row(pivot_col_index)

        # Normalize the pivot row
        self.dictionary[pivot_row_index, :] = self.dictionary[pivot_row_index, :] / -self.dictionary[pivot_row_index, pivot_col_index]
        
        # Row reduce all other rows so values above and below pivot are zero
        for j, row in enumerate(self.dictionary):
            if j != pivot_row_index:
                multiple = self.dictionary[j, pivot_col_index] / self.dictionary[pivot_row_index, pivot_col_index]
                self.dictionary[j, :] = self.dictionary[j, :] - multiple * self.dictionary[pivot_row_index, :]


    def solve(self):
        """Solve the linear optimization problem.

        Returns:
            (float) The minimum value of the objective function.
            (dict): The basic variables and their values.
            (dict): The nonbasic variables and their values.
        """
        # Ensure that the top row has negative values otherwise it is optimized
        while (self.dictionary[0, 1:] < 0).any():
            self.pivot()

        # Obtain minimum value in top corner
        minimum = self.dictionary[0, 0]

        dependant = {}
        independant = {}

        # Iterate through the dictionary selecting the dependant and independant variables
        for j, value in enumerate(self.dictionary[0, 1:]):
            if value != 0:
                dependant[int(j)] = float(0)
            else:
                for i, num in enumerate(self.dictionary[1:, j + 1]):
                    if num != 0:
                        independant[int(j)] = float(self.dictionary[i + 1, 0])
        
        return(minimum, independant, dependant)




In [4]:
def product_mix_solver(filename='productMix.npz'):
    """Solve the product mix problem for the data in 'productMix.npz'.

    Parameters:
        filename (str): the path to the data file.

    Returns:
        ((n,) ndarray): the number of units that should be produced for each product.
    """
    # Initialize the file
    file = np.load(filename)
    A = file['A']
    c = -file['p']
    m = file['m']
    d = file['d']

    # Create the rest of A with the production constaints
    A_stretch = np.eye(len(c))
    A = np.vstack((A, A_stretch))

    # Stack constraints and production constraints
    b = np.hstack((m, d))

    # Solve
    solver = SimplexSolver(c, A, b)
    sol = solver.solve()

    # Return the optimum of each product
    return np.array([sol[1][i] for i in [0, 1, 2, 3]])

We want to use the Simplex method to solve the **product mix problem** using linear programming.

Suppose a manufacturer produces \(n\) products using \(m\) different resources (labor, raw materials, machine time, etc.).  
- Product \(i\) sells at unit price \(p_i\).  
- There are at most \(m_j\) units available of resource \(j\).  
- Each unit of product \(i\) uses \(a_{j,i}\) units of resource \(j\).  
- The demand for product \(i\) is \(d_i\) units per time period.

The goal is to choose how much of each product to make in a given time period to maximize total revenue without using more than the available resources or exceeding demand.


We utilize the class we built above to solve this problem.


In [9]:
for i, value in enumerate(product_mix_solver(filename='productMix.npz')):
    print(f"For product {i + 1} we need {value} units.")


For product 1 we need 10.0 units.
For product 2 we need 6.192982456140348 units.
For product 3 we need 12.0 units.
For product 4 we need 1.7894736842105292 units.


/var/folders/4y/hv4lhv9j7d7gb28180kz91m00000gn/T/ipykernel_4455/2332329278.py:12: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  A = file['A']
/var/folders/4y/hv4lhv9j7d7gb28180kz91m00000gn/T/ipykernel_4455/2332329278.py:13: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  c = -file['p']
/var/folders/4y/hv4lhv9j7d7gb28180kz91m00000gn/T/ipykernel_4455/2332329278.py:14: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  m = file['m']
/var/folders/4y/hv4lhv9j7d7gb28180kz91m00000gn/T/ipykernel_4455/2332329278.py:15: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Pytho